In [ ]:
!pip install py7zr
import py7zr

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import time
import cv2
import numpy as np
import pandas as pd
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"; 

import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Model

In [ ]:
df_train=pd.read_csv('../input/data-csv/df_train.csv')
df_val=pd.read_csv('../input/data-csv/df_val.csv')
df_test=pd.read_csv('../input/data-csv/df_test.csv')

In [ ]:
archive=py7zr.SevenZipFile('../input/cifar-10/train.7z', mode='r')
archive.extractall(path="kaggle/working/temp")
archive.close()

In [ ]:
Image_Width=32
Image_Height=32
Image_Size=(Image_Width,Image_Height)
Image_Channels=3
batch_size=128

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.1, height_shift_range=0.1, fill_mode="nearest")
train_generator = train_datagen.flow_from_dataframe(df_train, directory="kaggle/working/temp/train/",x_col='id',y_col='label', shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size,)

validation_datagen = ImageDataGenerator(rescale=1./255)
val_generator = validation_datagen.flow_from_dataframe(df_val,  "kaggle/working/temp/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(df_test,  "kaggle/working/temp/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )


In [ ]:
model=tf.keras.models.load_model ('../input/data-csv/model_real.h5')

In [ ]:
predict = model.predict(train_generator)

In [ ]:
df_train['actual_lable'] =  train_generator.classes
df_train['predict'] = np.argmax(predict, axis=-1)
data_label=[]
for k,v in train_generator.class_indices.items():
  data_label.append(str(k))

In [ ]:

count=0
for i in range(len(df_train)):
    if df_train['actual_lable'][i]== df_train['predict'][i]:
        count+=1
print('so nhan dung la: ',count)


In [ ]:
# df_train=pd.read_csv('../input/data-csv/df_train_add_custom.csv')
# df_train=pd.read_csv('../input/data-csv/df_train_add_custom.csv')
# df_test=pd.read_csv('../input/data-csv/df_test_add_custom.csv')
data_train_addition_id=[]
data_train_addition_label=[]
for i in tqdm(range(len(df_train))):
    if (df_train['actual_lable'][i] !=df_train['predict'][i]):
        data_train_addition_id.append(df_train['id'][i])
        data_train_addition_label.append(df_train['label'][i])
    elif np.max(predict[i])<0.78:
        data_train_addition_id.append(df_train['id'][i])
        data_train_addition_label.append(df_train['label'][i])
df_aug=pd.DataFrame()
df_aug['id']=pd.DataFrame(data_train_addition_id)
df_aug['label']=pd.DataFrame(data_train_addition_label)
df_aug.to_csv('wrong_train_78.csv',index=False)
    

In [ ]:
df_val.to_csv('df_val_add_custom.csv',index=False)

# visualize dataset

In [ ]:
count=0
for k in range(0,10):
    for i in tqdm(range(len(df_test))):
        if (df_test['actual_lable'][i] !=df_test['predict'][i]):
            if (df_test['actual_lable'][i]==k):
                count+=1
    print(k,count)
    count=0

count=0
print('TRAINING')
for k in range(0,10):
    for i in tqdm(range(len(df_train))):
        if (df_train['actual_lable'][i] !=df_train['predict'][i]):
            if (df_train['actual_lable'][i]==k):
                count+=1
    print(k,count)
    count=0
print('VALIDATION')
for k in range(0,10):
    for i in tqdm(range(len(df_val))):
        if (df_val['actual_lable'][i] !=df_val['predict'][i]):
            if (df_val['actual_lable'][i]==k):
                count+=1
    print(k,count)
    count=0

In [ ]:
wrong_train_03=pd.read_csv('./wrong_train_78.csv')
wrong_val_03=pd.read_csv('./wrong_val_78.csv')
wrong_test_03=pd.read_csv('./wrong_test_78.csv')
print(len(wrong_train_03))
print(len(wrong_val_03))
print(len(wrong_test_03))

In [ ]:
a=[]
for i in range(len(df_test)):
    if df_test['actual_lable'][i]== df_test['predict'][i]:
        a.append(np.max(predict[i],axis=-1))
print(np.min(a))

# Làm giàu dữ liệu

In [ ]:
wrong_train_03=pd.read_csv('./wrong_train_78.csv')
wrong_val_03=pd.read_csv('./wrong_val_78.csv')
wrong_test_03=pd.read_csv('./wrong_test_78.csv')

In [ ]:
data_train_addition_id=[]
data_train_addition_label=[]

In [ ]:
data_train_addition_id

In [ ]:
path_dir_save='kaggle/working/temp2/train/'
from tqdm import tqdm
for i in tqdm(range(len(wrong_train_03))):
    fname=wrong_train_03['id'][i]
    img=cv2.imread(os.path.join('kaggle/working/temp2/train/',fname))
    rotated_img=cv2.flip(img, 1)
    rotated_img_name=fname[:(len(fname)-4)] + '_flip_'+fname[(len(fname)-4):]
    data_train_addition_id.append(rotated_img_name)
    data_train_addition_label.append(wrong_train_03['label'][i])
    cv2.imwrite(os.path.join(path_dir_save, rotated_img_name),rotated_img)

In [ ]:
wrong_train_032=pd.DataFrame()
wrong_train_032['id']=pd.DataFrame(data_train_addition_id)
wrong_train_032['label']=pd.DataFrame(data_train_addition_label)
wrong_train_03=pd.concat([wrong_train_03,wrong_train_032],ignore_index=True)

In [ ]:
data_train_addition_id=[]
data_train_addition_label=[]

In [ ]:
goc=5
path_dir_save='kaggle/working/temp2/train/'
for i in tqdm(range(len(wrong_train_03))):
    fname=str(wrong_train_03['id'][i])
#     print(len(fname))
#     print(os.path.join('kaggle/working/temp3/train/',fname))
    img=cv2.imread(os.path.join('kaggle/working/temp2/train/',fname))
    h,w=img.shape[:2]
    rotation_matrix=cv2.getRotationMatrix2D((w/2,h/2),goc,.8)
    rotated_img=cv2.warpAffine(img,rotation_matrix,(w,h))
    rotated_img_name=fname[:(len(fname)-4)] + '_rotated_'+str(goc)+fname[(len(fname)-4):]
    data_train_addition_id.append(rotated_img_name)
    data_train_addition_label.append(wrong_train_03['label'][i])
    cv2.imwrite(os.path.join(path_dir_save, rotated_img_name),rotated_img)

In [ ]:
df_train_062=pd.DataFrame()
df_train_062['id']=pd.DataFrame(data_train_addition_id)
df_train_062['label']=pd.DataFrame(data_train_addition_label)
# df_train_06=pd.concat([df_train_06,df_train_062],ignore_index=True)
wrong_train_03=pd.concat([wrong_train_03,df_train_062],ignore_index=True)

In [ ]:
Image_Width=71
Image_Height=71
Image_Size=(Image_Width,Image_Height)
Image_Channels=3
batch_size=128

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.1, height_shift_range=0.1, fill_mode="nearest")
train_generator = train_datagen.flow_from_dataframe(wrong_train_03, directory="kaggle/working/temp/train/",x_col='id',y_col='label', target_size=Image_Size, class_mode='categorical', batch_size=batch_size,)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(wrong_val_03,  "kaggle/working/temp/train/",x_col='id',y_col='label',target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(wrong_test_03,  "kaggle/working/temp/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )


In [ ]:

# Build model
base_model = tf.keras.applications.xception.Xception(include_top=False,
                                                     weights='imagenet',
                                                     input_shape = (71,71, 3),
                                                     pooling = "max"
                                                    )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(32, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(10, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.00001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint,EarlyStopping

checkpoint=ModelCheckpoint('model_xception_78_2acc.h5',
                           monitor="val_accuracy",
                           verbose=0,
                           save_best_only=True,
                           save_weights_only=False,
                           mode="auto",
                           save_freq="epoch")
checkpoint2=ModelCheckpoint('model_xception_78_2loss.h5',
                           monitor="val_loss",
                           verbose=0,
                           save_best_only=True,
                           save_weights_only=False,
                           mode="auto",
                           save_freq="epoch")
earlystop = EarlyStopping(monitor="val_accuracy",
                          min_delta=0,
                          mode="auto",
                          patience=50,
                          restore_best_weights=True,)
earlystop2 = EarlyStopping(monitor="val_loss",
                          min_delta=0,
                          mode="auto",
                          patience=50,
                          restore_best_weights=True,)
callbacks = [checkpoint,checkpoint2,earlystop,earlystop2]

In [ ]:
n_epochs = 300
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = callbacks)

In [ ]:
H=history
# PLOT MODEL HISTORY OF ACCURACY AND LOSS OVER EPOCHS
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('Xception_03_Model_Accuracy.png')
plt.show()
# summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('Xception_03_Model_Loss.png')
plt.show()

# Evaluate

In [ ]:
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_dataframe(df_test_06,  "kaggle/working/temp/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(df_test,  "kaggle/working/temp/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

In [ ]:
model=tf.keras.models.load_model ('./model_xception_78_2acc.h5')

In [ ]:
predict = model.predict(test_generator)

In [ ]:
# df_test_06['actual_lable'] =  test_generator.classes
# df_test_06['predict'] = np.argmax(predict, axis=-1)
# data_label=[]
# for k,v in test_generator.class_indices.items():
#   data_label.append(str(k))

df_test['actual_lable'] =  test_generator.classes
df_test['predict'] = np.argmax(predict, axis=-1)
data_label=[]
for k,v in test_generator.class_indices.items():
  data_label.append(str(k))

In [ ]:

# from sklearn.metrics import classification_report
# print(classification_report(df_test_06['actual_lable'], df_test_06['predict'], target_names = data_label))
count1=0
count2=0
for i in range(len(df_test)):
    if wrong_test_03['actual_lable'][i]== np.argmax(predict[i], axis=-1):
        count1+=1
    else:
        count2+=1
print(count1,count2)
from sklearn.metrics import classification_report
print(classification_report(df_test['actual_lable'], df_test['predict'], target_names = data_label))



# Predict

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import time
import cv2

#Load test data set
df_test=pd.read_csv('../input/data-csv/df_test.csv')

# Define parameters
Image_Width=32
Image_Height=32
Image_Size=(Image_Width,Image_Height)
Image_Channels=3
batch_size=128
#Processing data
print('Processing data ...')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(df_test,  "kaggle/working/temp/train/",\
    x_col='id',y_col='label',shuffle=False,target_size=Image_Size, \
        class_mode='categorical', batch_size=batch_size )

Image_Width=71
Image_Height=71
Image_Size=(Image_Width,Image_Height)
Image_Channels=3
batch_size=128
#Processing data
print('Processing data ...')
test_generator2 = test_datagen.flow_from_dataframe(df_test,  "kaggle/working/temp/train/",\
    x_col='id',y_col='label',shuffle=False,target_size=Image_Size, \
        class_mode='categorical', batch_size=batch_size )

In [ ]:
# Load model
print('Loading model ...')
model=keras.models.load_model("../input/data-csv/model_real.h5")
model2=keras.models.load_model('../input/data-csv/best_weights_xception_acc_model.h5')
#Predict 
print('Predicting data ...')
start=time.time()
predict1 = model.predict(test_generator)
time2=time.time()
print(' Time predict :',time2-start ,'s')
predict2 = model2.predict(test_generator2)
end=time.time()
print(' Time predict :',end-time2 ,'s')

In [ ]:
count1=0
count2=0
count3=0
count4=0
predict=[]
df_test['actual_lable'] =  test_generator.classes
for i in range(len(df_test)):
    if df_test['actual_lable'][i]== np.argmax(predict1[i], axis=-1):
        count1+=1
        if np.max(predict1[i]) >= 0.3:
            count2+=1
            predict.append(np.argmax(predict1[i]))
        elif df_test['actual_lable'][i]== np.argmax(predict2[i], axis=-1):
            count3+=1
            predict.append(np.argmax(predict2[i]))
        else:
            count4+=1
    elif df_test['actual_lable'][i]== np.argmax(predict2[i], axis=-1):
        count3+=1
    else:
        count4+=1
print('Số ảnh dự đoán đúng của model custom là: ',count2)
print('Số ảnh dự đoán đúng của model xception là: ',count3)
print('Số ảnh dự đoán sai là: ',count4)
count1,count2,count3,count4,(count2+count3)/5000

In [ ]:
len(predict)

In [ ]:
predict=[]
for i in range(len(df_test)):
  if np.max(predict1[i])>=0.3:
    predict.append(np.argmax(predict1[i], axis=-1))
  else:
    predict.append(np.argmax(predict2[i], axis=-1))

#Etestuate model
df_test['actual_lable'] =  test_generator.classes
# df_test['predict'] = np.argmax(predict, axis=-1)
df_test['predict'] =pd.DataFrame(predict)
count=0
for i in range(len(df_test)):
  if df_test['actual_lable'][i]== df_test['predict'][i]:
    count+=1
print('so anh du doan dung la: ',count)
print('ACC: ',count/5000)
data_label=[]
for k,v in test_generator.class_indices.items():
  data_label.append(str(k))
##F1 score
from sklearn.metrics import classification_report
print(classification_report(df_test['actual_lable'], \
    df_test['predict'], target_names = data_label))
##Confusion matrix
# CREATE CONFUSION MATRIX OF ACTUAL VS. PREDICTION 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_test['actual_lable'], df_test['predict'])
plt.figure(figsize = (8, 5))
cm = pd.DataFrame(cm , index = data_label , columns = data_label)
import seaborn as sns
ax = sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.savefig('Initial_Model_Confusion_Matrix.png')
plt.show()
print(0)